## Homework

The goal of this homework is to train a simple model for predicting the duration of a ride - similar to what we did in this module.


In [26]:
import os
import pandas as pd
import itertools
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error, root_mean_squared_error

In [4]:
!pip install pyarrow


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


## Q1. Downloading the data


We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

* 16
* 17
* 18
* 19

In [5]:
DATASET_DIR = "data"
TRAIN_FILE = 'yellow_tripdata_2023-01.parquet'
VAL_FILE   = 'yellow_tripdata_2023-02.parquet'
DATA_URL = 'https://d37ci6vzurychx.cloudfront.net/trip-data'

In [6]:
if not os.path.isdir(DATASET_DIR): 
    
    # if the directory is  
    # not present then create it. 
    os.makedirs(DATASET_DIR)
    print(f"The {DATASET_DIR} directory is created")

The data directory is created


In [7]:
# Donwloading datasets
downloading_file = [TRAIN_FILE, VAL_FILE]
for fdata in downloading_file:
    data_path = os.path.join(DATASET_DIR, fdata)
    if os.path.isfile(data_path):
        print(f"The {fdata} file exists")
        continue
    print(f"Downloading file {fdata} from {DATA_URL}/{fdata}")
    !curl -o {data_path} {DATA_URL}/{fdata}
    print(f"The {fdata} file is downloaded")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 45.4M  100 45.4M    0     0   270M      0 --:--:-- --:--:-- --:--:--  270M
The yellow_tripdata_2023-01.parquet file is downloaded
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 45.5M  100 45.5M    0     0   385M      0 --:--:-- --:--:-- --:--:--  385M
The yellow_tripdata_2023-02.parquet file is downloaded


In [8]:
TRAIN_DATA_PATH = f'{DATASET_DIR}/{TRAIN_FILE}'
VAL_DATA_PATH   = f'{DATASET_DIR}/{VAL_FILE}'

In [9]:
# Read train data
print(f"Reading {TRAIN_DATA_PATH}")
train_df = pd.read_parquet(TRAIN_DATA_PATH)
train_df.head(10)

Reading data/yellow_tripdata_2023-01.parquet


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00
5,2,2023-01-01 00:50:34,2023-01-01 01:02:52,1.0,1.84,1.0,N,161,137,1,12.8,1.00,0.5,10.00,0.0,1.0,27.80,2.5,0.00
6,2,2023-01-01 00:09:22,2023-01-01 00:19:49,1.0,1.66,1.0,N,239,143,1,12.1,1.00,0.5,3.42,0.0,1.0,20.52,2.5,0.00
7,2,2023-01-01 00:27:12,2023-01-01 00:49:56,1.0,11.70,1.0,N,142,200,1,45.7,1.00,0.5,10.74,3.0,1.0,64.44,2.5,0.00
8,2,2023-01-01 00:21:44,2023-01-01 00:36:40,1.0,2.95,1.0,N,164,236,1,17.7,1.00,0.5,5.68,0.0,1.0,28.38,2.5,0.00
9,2,2023-01-01 00:39:42,2023-01-01 00:50:36,1.0,3.01,1.0,N,141,107,2,14.9,1.00,0.5,0.00,0.0,1.0,19.90,2.5,0.00


In [12]:
# Print shape
train_df.shape

(3066766, 19)

Read the data for January. How many columns are there?

* 16
* 17
* 18
* **19**

## Q2. Computing duration
Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

* 32.59
* 42.59
* 52.59
* 62.59

In [13]:
# Calculate duration
#train_df['duration'] = (train_df.tpep_dropoff_datetime - train_df.tpep_pickup_datetime) / pd.Timedelta(minutes=1)
train_df['duration'] = train_df.tpep_dropoff_datetime - train_df.tpep_pickup_datetime
train_df.duration = train_df.duration.apply(lambda td: td.total_seconds() / 60)


In [14]:
train_df[['duration']].head(5)

,duration
0,8.433333
1,6.316667
2,12.750000
3,9.616667
4,10.833333


In [15]:
train_df[['duration']].std()

duration    42.594351
dtype: float64

What's the standard deviation of the trips duration in January?

* 32.59
* **42.59**
* 52.59
* 62.59

## Q3. Dropping outliers


Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

* 90%
* 92%
* 95%
* 98%

In [17]:
duration_filter = (train_df['duration'] >= 1) & (train_df['duration'] <= 60)
filtered_train_df = train_df[duration_filter]
filtered_train_df.shape

(3009173, 20)

In [18]:
(filtered_train_df.shape[0]*100)/train_df.shape[0]

98.1220282212598

What fraction of the records left after you dropped the outliers?

* 90%
* 92%
* 95%
* **98%**

## Q4. One-hot encoding


Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
Fit a dictionary vectorizer
Get a feature matrix from it
What's the dimensionality of this matrix (number of columns)?

* 2
* 155
* 345
* 515
* 715

In [19]:
categorical = ['PULocationID', 'DOLocationID']

filtered_train_df[categorical] = filtered_train_df[categorical].astype(str)

train_dicts = filtered_train_df[categorical].to_dict(orient='records')

/tmp/ipykernel_12774/2040271735.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_train_df[categorical] = filtered_train_df[categorical].astype(str)


In [20]:
#dict(itertools.islice(train_dicts.items(), 5))
train_dicts[:10]

[{'PULocationID': '161', 'DOLocationID': '141'},
 {'PULocationID': '43', 'DOLocationID': '237'},
 {'PULocationID': '48', 'DOLocationID': '238'},
 {'PULocationID': '138', 'DOLocationID': '7'},
 {'PULocationID': '107', 'DOLocationID': '79'},
 {'PULocationID': '161', 'DOLocationID': '137'},
 {'PULocationID': '239', 'DOLocationID': '143'},
 {'PULocationID': '142', 'DOLocationID': '200'},
 {'PULocationID': '164', 'DOLocationID': '236'},
 {'PULocationID': '141', 'DOLocationID': '107'}]

In [21]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [22]:
# Print dimensions
X_train.shape

(3009173, 515)

What's the dimensionality of this matrix (number of columns)?

* 2
* 155
* 345
* **515**
* 715

## Q5. Training a model

Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters, where duration is the response variable
Calculate the RMSE of the model on the training data
What's the RMSE on train?

* 3.64
* 7.64
* 11.64
* 16.64

In [23]:
# Get y value
target = 'duration'
y_train = filtered_train_df[target].values

In [25]:
# Train model
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

In [27]:
root_mean_squared_error(y_train, y_pred)

7.6492624397080675

Train a plain linear regression model with default parameters, where duration is the response variable
Calculate the RMSE of the model on the training data
What's the RMSE on train?

* 3.64
* **7.64**
* 11.64
* 16.64

## Q6. Evaluating the model


Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

* 3.81
* 7.81
* 11.81
* 16.81

In [28]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [29]:
# Read validation data
print(f"Reading {VAL_DATA_PATH}")
df_val = read_dataframe(VAL_DATA_PATH)
df_val.head(10)

Reading data/yellow_tripdata_2023-02.parquet


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,duration
0,1,2023-02-01 00:32:53,2023-02-01 00:34:34,2.0,0.30,1.0,N,142,163,2,4.4,3.50,0.5,0.00,0.0,1.0,9.40,2.5,0.00,1.683333
3,1,2023-02-01 00:29:33,2023-02-01 01:01:38,0.0,18.80,1.0,N,132,26,1,70.9,2.25,0.5,0.00,0.0,1.0,74.65,0.0,1.25,32.083333
4,2,2023-02-01 00:12:28,2023-02-01 00:25:46,1.0,3.22,1.0,N,161,145,1,17.0,1.00,0.5,3.30,0.0,1.0,25.30,2.5,0.00,13.300000
5,1,2023-02-01 00:52:40,2023-02-01 01:07:18,1.0,5.10,1.0,N,148,236,1,21.9,3.50,0.5,5.35,0.0,1.0,32.25,2.5,0.00,14.633333
6,1,2023-02-01 00:12:39,2023-02-01 00:40:36,1.0,8.90,1.0,N,137,244,1,41.5,3.50,0.5,3.50,0.0,1.0,50.00,2.5,0.00,27.950000
7,1,2023-02-01 00:56:53,2023-02-01 01:00:37,1.0,1.20,1.0,N,263,141,1,7.2,3.50,0.5,2.44,0.0,1.0,14.64,2.5,0.00,3.733333
8,2,2023-02-01 00:20:40,2023-02-01 00:33:56,1.0,7.49,1.0,N,48,243,1,30.3,1.00,0.5,8.82,0.0,1.0,44.12,2.5,0.00,13.266667
9,2,2023-02-01 00:33:51,2023-02-01 00:37:34,1.0,0.51,1.0,N,114,211,1,5.8,1.00,0.5,1.62,0.0,1.0,12.42,2.5,0.00,3.716667
10,2,2023-02-01 01:00:45,2023-02-01 01:06:00,1.0,0.78,1.0,N,114,249,1,7.2,1.00,0.5,2.44,0.0,1.0,14.64,2.5,0.00,5.250000
11,2,2023-02-01 00:10:48,2023-02-01 00:18:09,1.0,1.57,1.0,N,125,107,1,10.0,1.00,0.5,1.00,0.0,1.0,16.00,2.5,0.00,7.350000


In [30]:
val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)
# Get validation y value
target = 'duration'
y_val = df_val[target].values

In [31]:
# Validate model
y_pred = lr.predict(X_val)

In [32]:
root_mean_squared_error(y_val, y_pred)

7.81181211389241

What's the RMSE on validation?

* 3.81
* **7.81**
* 11.81
* 16.81

## Submit the results

* Submit your results here: https://courses.datatalks.club/mlops-zoomcamp-2025/homework/hw1
* If your answer doesn't match options exactly, select the closest one